In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, chi2

In [9]:
df = pd.read_csv('swat.csv')
#df.shape

#df = df.iloc[:5*len(df.index)//8]
#df = df.iloc[:len(df.index)//2]
df = df.iloc[:7500]
y = df["Normal/Attack"]
del df["Normal/Attack"]
df.tail(5)

,timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
7495,1451304295000000000,2.583352,510.4036,2,2,1,263.4261,8.370802,329.5309,2.442828,...,0.306505,2,1,249.5514,1.778105,188.5734,1.737353,1,2,1
7496,1451304296000000000,2.589117,510.2073,2,2,1,263.4261,8.370802,329.5309,2.444879,...,0.306633,2,1,249.5514,1.778105,188.6535,1.737161,1,2,1
7497,1451304297000000000,2.592000,510.0896,2,2,1,263.4261,8.370802,329.5309,2.444879,...,0.306953,2,1,249.6635,1.778105,188.6535,1.736584,1,2,1
7498,1451304298000000000,2.592000,510.1288,2,2,1,263.4261,8.370802,329.5309,2.444879,...,0.306953,2,1,249.6635,1.778105,188.5254,1.736584,1,2,1
7499,1451304299000000000,2.592000,509.7363,2,2,1,263.4261,8.368238,329.5309,2.444879,...,0.306889,2,1,249.6635,1.778105,188.5254,1.736584,1,2,1


In [10]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """Aggregate values over given time span.
    Args:
        X (ndarray or pandas.DataFrame):
            N-dimensional sequence of values.
        interval (int):
            Integer denoting time span to compute aggregation of.
        time_column (int):
            Column of X that contains time values.
        method (str or list):
            Optional. String describing aggregation method or list of strings describing multiple
            aggregation methods. If not given, `mean` is used.
    Returns:
        ndarray, ndarray:
            * Sequence of aggregated values, one column for each aggregation method.
            * Sequence of index values (first index of each aggregated segment).
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)

    X = X.sort_values(time_column).set_index(time_column)

    if isinstance(method, str):
        method = [method]

    start_ts = X.index.values[0]
    max_ts = X.index.values[-1]

    values = list()
    index = list()
    while start_ts <= max_ts:
        end_ts = start_ts + interval
        subset = X.loc[start_ts:end_ts]
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method
        ]
        values.append(np.concatenate(aggregated))
        index.append(start_ts)
        start_ts = end_ts

    return np.asarray(values), np.asarray(index)

X_tsa, index = time_segments_aggregate(df, interval=1000000000, time_column='timestamp')

In [11]:
imp = SimpleImputer()
X_imp = imp.fit_transform(X_tsa)

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1)) ## for the gradients to converge faster == try 0-1 StandardScaler()
X_scl = scaler.fit_transform(X_imp)
X_scl.shape

(7500, 51)

In [13]:
y = np.array(y, dtype=str)
y_binary = []
for i in y:
    if i == "Normal":
        y_binary.append(0)
    else:
        y_binary.append(1)
        
y_binary = np.array(y_binary)
y_binary

array([0, 0, 0, ..., 0, 0, 0])

In [17]:
select = SelectKBest(chi2, k=20)
X_chi2 = select.fit_transform(X_scl, y)
X_chi2[0]

array([9.07118188e-01, 7.94695400e-02, 1.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 9.69911906e-01, 7.00800432e-01, 8.71841595e-01,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 7.51658934e-01,
       7.36774901e-01, 7.31694322e-01, 3.75810794e-01, 3.13731489e-01,
       7.70974266e-01, 7.27519061e-01, 7.36975463e-05, 0.00000000e+00])

In [34]:
features = np.array(df.columns, dtype=str)
features

array(['timestamp', 'FIT101', 'LIT101', ' MV101', 'P101', 'P102',
       ' AIT201', 'AIT202', 'AIT203', 'FIT201', ' MV201', ' P201',
       ' P202', 'P203', ' P204', 'P205', 'P206', 'DPIT301', 'FIT301',
       'LIT301', 'MV301', 'MV302', ' MV303', 'MV304', 'P301', 'P302',
       'AIT401', 'AIT402', 'FIT401', 'LIT401', 'P401', 'P402', 'P403',
       'P404', 'UV401', 'AIT501', 'AIT502', 'AIT503', 'AIT504', 'FIT501',
       'FIT502', 'FIT503', 'FIT504', 'P501', 'P502', 'PIT501', 'PIT502',
       'PIT503', 'FIT601', 'P601', 'P602', 'P603'], dtype='<U9')

In [42]:
len(features)

52

In [43]:
filter = select.get_support()
len(filter)

51

In [47]:
selected_features = []
for i in range(len(features)-2):
    print(i)
    if filter[i]:
        selected_features.append(features[i+1])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
